# Animal Game (_Jogo do Bicho_)

The Animal Game is a popular lottery-like game played in Brazil since the 1940s,
despite being ilegal for most of its existence, up until this day.

Every **lottery draw** contains five numbers with four digits each, from '0000' to '9999' and
players can make different kinds of bets. 

The most **common bet** is to pick one out of 25 possible animals. Each animal corresponds to
a group of four numbers.

The first animal is the Ostrich (_Avestruz_), and its numbers are: [01, 02, 03, 04], the next
is the Eagle (_Águia_) [05, 06, 07, 08], and so forth, up until the Cow (_Vaca_) [97, 98, 99, 00].

A player who bets on a single animal **wins** if any of the animals four numbers appear in the
last two digits of any of the five lottery draws. There are many other kinds of bets.

For instance, the same game described above can be played in 'dry' or 'head' mode, where the
player only wins if the chosen animal appears in the first out of the five draws. This is five
times less likely so the payout of this game mode is usually five times greater than the latter.

Players can also bet on specific numbers instead groups of numbers (animals). The bets are made
on the last two, three or four digits of the draws. If either of these bets are made in 'dry'
mode only the first draw of the lot is considered and the prize is five times greater.

Finally it is also possible to bet on two or three animals, or even 2 double digit numbers.


## What is this thing anyway?

- A bad excuse to use metaclasses. 
- A mini framework for easily creating different types of games and running simulations of the Animal Game.
- A random act of data exploration.

In [1]:
ANIMALS = ('Avestruz', 'Águia', 'Burro', 'Borboleta', 'Cachorro', 'Cabra', 'Carneiro', 'Camelo', 'Cobra', 'Coelho', 'Cavalo', 'Elefante', 'Galo', 'Gato', 'Jacaré', 'Leão', 'Macaco', 'Porco', 'Pavão', 'Peru', 'Touro', 'Tigre', 'Urso', 'Veado', 'Vaca',)

In [2]:
def animal_id_from_digits(digits: str) -> str:
    i = (int(digits[-2:])-1)//4
    if i < 0:
        i = 24
    return str(i).zfill(2)

The **Round class** is used to compare a bet choice and a draw.

The **animal flag** is used to indicate wether the choices represent the index of an animal or a two digit number. This is necessary because both are represented in the same way: with a two digit zero padded number (as a string).

In [3]:
from collections import Counter

class Round:

    def __init__(self, choices: list, draw: list, animal=False):
        self.choices = choices
        if animal:
            self.draw = [animal_id_from_digits(d) for d in draw]
        else:
            self.draw = [d[-len(choices[0]):] for d in draw]

    def check_present(self) -> bool:
        prev_count = sum(Counter(self.draw).values())
        after_removal = sum( (Counter(self.draw) - Counter(self.choices)).values() )
        return prev_count - after_removal == len(self.choices)
    
    def check_ordered(self) -> bool:
        length = len(self.choices)
        try:
            first_idx = self.draw.index(self.choices[0])
        except ValueError:
            return False
        return self.choices == self.draw[first_idx:first_idx+length]
    
    def check_exact(self) -> bool:
        matches = [self.choices[i] == self.draw[i] for i in range(len(self.choices))]
        return all(matches)

**GameMeta** is a metaclass.

- It probably did not have to be a metaclass, but it is.
- I blame [David Beazley](https://youtu.be/sPiWg5jSoZI)

The metaclass will inject into the class the **calc**, **check_win** and
**random** methods. It will also create one instance of the class
and keep it in a dictionary.

> The idea behind this madness is to permit the creation of different kinds of games by simply declaring a class that contains class variables that describe how the game should work.

The metaclass also provides convinience methods to list all game type instances
or to grab one by the name of the class.

In [4]:
import numpy as np

class GameMeta(type):

    __type_instances = {}

    def __new__(cls, name, bases, clsdict):

        # Create the class:
        clsobj = super().__new__(cls, name, bases, clsdict)
        
        # Set default class vars if they are not present:
        if not hasattr(clsobj, 'prize'):
            setattr(clsobj, 'prize', 2)
        if not hasattr(clsobj, 'match_index'):
            setattr(clsobj, 'match_index', False)
        if not hasattr(clsobj, 'match_order'):
            setattr(clsobj, 'match_order', False)
        if not hasattr(clsobj, 'n_choices'):
            setattr(clsobj, 'n_choices', 1)
        if not hasattr(clsobj, 'len_choice'):
            setattr(clsobj, 'len_choice', 2)
        if not hasattr(clsobj, 'animal'):
            setattr(clsobj, 'animal', False)
        if not hasattr(clsobj, 'desc'):
            setattr(clsobj, 'desc', 'missing desc')
            
        # Do some validation of the variables:
        if clsobj.len_choice < 1 or clsobj.len_choice > 4:
            raise TypeError(f'{name}: len_choice must be at least 1, at most 4')
        if clsobj.n_choices < 1 or clsobj.n_choices > 5:
            raise TypeError(f'{name}: n_choices must be at least 1, at most 5')
        if clsobj.animal and clsobj.len_choice != 2:
            raise TypeError(f'{name}: Conflicting settings: animal=True implies len_choice=2')
        if clsobj.match_index and clsobj.match_order:
            print(f'{name} (warning): match_index=True makes match_order redundant')

        # Declare the inner functions:
        
        def check_win(self, choices: list, draw: list) -> bool:
            if clsobj.n_choices != len(choices):
                raise TypeError(f'Expected {clsobj.n_choices} choices, got {len(choices)}')
            if not all([len(c) == clsobj.len_choice for c in choices]):
                raise TypeError(f'Choices must be all of length {clsobj.len_choice}')
            _draw =[d[-(clsobj.len_choice):] for d in draw]
            game_round = Round(choices=choices, draw=draw, animal=clsobj.animal)
            if clsobj.match_index:
                return game_round.check_exact()
            elif clsobj.match_order:
                return game_round.check_ordered()
            else:
                return game_round.check_present()

        def calc(self, bet: float, choices: list, draw: list) -> float:
            if self.check_win(choices, draw):
                return (bet * clsobj.prize) - bet
            else:
                return -1 * bet

        def random(self) -> list:
            '''Returns random picks for the corresponding game type'''
            high = 24 if clsobj.animal else 10**clsobj.len_choice-1
            r = np.random.randint(low=0, high=high, size=clsobj.n_choices)
            return [str(v).zfill(clsobj.len_choice) for v in r]

        # End inner functions. Inject them into the class:
        setattr(clsobj, 'random', random)
        setattr(clsobj, 'check_win', check_win)
        setattr(clsobj, 'calc', calc)
        setattr(clsobj, 'name', name)
        
        # Create one instance of the class and set it in a dict where
        # the class name is the key and the instance the value:
        GameMeta.__type_instances[name] = clsobj()
        
        # Return the class from the metaclass so the universe doesn't break:
        return clsobj
    
    @classmethod
    def get_type(cls, name):
        return cls.__type_instances.get(name)
    
    @classmethod
    def get_types(cls):
        return tuple(cls.__type_instances.values())

## Declare new game types

Bellow is an example of how the metaclass is supposed to be used. The games
listed bellow are the most common games in the Animal Game.

Many more can be created by combining the settings.
A few settings conflict with each other, but hopefully the metaclass will
detect this and produce a warning or error as appropriate.


### Settings:

- **desc**: _str_
    - description or name of the type of game.


- **animal**: _bool_
    - Is this bet on an animal? If not it is on a number.


- **len_choice**: _int_
    - The number of digits in each choice. Bets on animals always have two digits because animals are represented by a zero padded two digit string.


- **n_choices**: _int_
    - The number of picks for this kind of game.


- **match_order**: _bool_
    - If true not only must the picks be present in the draw, they must also be appear in the same order. This setting has no effect if n_choices=1.


- **match_index**: _bool_
    - More strict than match_order, this one requires that the exact position of the picks be matched, not only the order. Overrides match_order.


- **prize**: _float_
    - The payout to the player is calculated by multiplying the amount bet for this value in case of a win.

In [5]:
# DEFAULTS (set by GameMeta if not present):
#     desc = 'missing desc'
#     animal = False
#     match_order = False
#     match_index = False
#     n_choices = 1
#     len_choice = 2
#     prize = 2

class DQ(metaclass=GameMeta):
    desc = 'Dezena Qualquer'
    prize = 12.0

class DS(metaclass=GameMeta):
    desc = 'Dezena Seca'
    prize = 60.0
    match_index = True

class GQ(metaclass=GameMeta):
    desc = 'Grupo Qualquer'
    animal = True
    prize = 3.6

class GS(metaclass=GameMeta):
    desc = 'Grupo Seco'
    animal = True
    match_index = True
    prize = 18.0

class GD(metaclass=GameMeta):
    desc = 'Grupo Duque'
    animal = True
    prize = 18.5
    n_choices = 2

class GT(metaclass=GameMeta):
    desc = 'Grupo Terno'
    animal = True
    n_choices = 3
    prize = 130.0

class DD(metaclass=GameMeta):
    desc = 'Duque Dezenas'
    prize = 300.0
    n_choices = 2

class DT(metaclass=GameMeta):
    desc = 'Dezena Terno'
    n_choices = 3
    prize = 3000.0

class CQ(metaclass=GameMeta):
    desc = 'Centena Qualquer'
    len_choice = 3
    prize = 120.0

class CS(metaclass=GameMeta):
    desc = 'Centena Seca'
    len_choice = 3
    match_index = True
    prize = 600.0

class MQ(metaclass=GameMeta):
    desc = 'Milhar Qualquer'
    len_choice = 4
    prize = 800.0

class MS(metaclass=GameMeta):
    desc = 'Milhar Seco'
    len_choice = 4
    match_index = True
    prize = 4000.0


# Example of a made up kind of game:

#class TI(metaclass=GameMeta):
#    desc = 'Terno Improvável'
#    animal = True
#    n_choices = 3
#    match_order = True
#    prize = 1200.0

In [6]:
def bet_sample(mean: float, std: float, size: int) -> np.array:
    '''Return a normally distributed sample of floats'''
    a = np.random.normal(loc=mean, scale=std, size=size)
    sample = np.around(a, decimals=1)
    return sample[sample > 0]

In [7]:
def random_draw() -> list:
    '''Returns a list of five strings of digits of length 4'''
    return [str(n).zfill(4) for n in np.random.randint(low=0, high=9999, size=5)]

In [8]:
# House point of view or player point of view?
# If true bets are money in (+) and payouts are money out (-)

house_pov = True

In [9]:
# TODO: Make this function faster

import pandas as pd

def make_simulation(bet_mean: float, bet_std: float, size: int, house_pov=True, draw=None) -> pd.DataFrame:
    '''Make a simulation of {size} bets with the game types declared as classes from GameMeta'''
    df = pd.DataFrame()
    game_types = GameMeta.get_types()
    df['bet'] = bet_sample(mean=bet_mean, std=bet_std, size=size)
    if draw is None:
        draw = random_draw()
    picks = []
    types = []
    codes = []
    net = []
    for bet in df['bet']:
        game = np.random.choice(game_types)
        types.append(game.desc)
        codes.append(game.name)
        pick = game.random()
        v = game.calc(bet=bet, draw=draw, choices=pick)
        if house_pov:
            v *= -1
        if game.animal:
            pick = [ANIMALS[int(i)] for i in pick]
        picks.append(pick)
        net.append(v)
    df['game_type'] = types
    df['code'] = codes
    df['picks'] = picks
    df['net'] = net
    df['draw'] = [draw for _ in range(len(df))]
    if not house_pov:
        df['bet'] = [-b for b in df['bet']]
    return df

In [10]:
import locale
locale.setlocale( locale.LC_ALL, '')

'LC_CTYPE=en_US.UTF-8;LC_NUMERIC=pt_BR.UTF-8;LC_TIME=en_US.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=pt_BR.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=pt_BR.UTF-8;LC_NAME=pt_BR.UTF-8;LC_ADDRESS=pt_BR.UTF-8;LC_TELEPHONE=pt_BR.UTF-8;LC_MEASUREMENT=pt_BR.UTF-8;LC_IDENTIFICATION=pt_BR.UTF-8'

In [11]:
def regroup_by_type(source, house_pov=True):
    df = pd.DataFrame()
    nets = source[['code', 'net']].groupby(['code']).sum()
    df['code'] = nets.index
    df['game_type'] = [GameMeta.get_type(s).desc for s in df['code']]
    df['n'] = source.groupby(['code']).count()['net'].values
    if house_pov:
        wins = source[source['net'] < 0].groupby(['code']).count()['picks']
    else:
        wins = source[source['net'] > 0].groupby(['code']).count()['picks']
    df['wins'] = [wins.get(k, 0) for k in df['code'].values]
    probs = [(a / b) * 100 for a, b in zip(df['wins'].values, df['n'].values)]
    df['p'] = [f'{prob:.3f}%' for prob in probs]
    df['net'] = [s for s in nets['net']]
    df.sort_values('net', inplace=True)
    avg = (df['wins'].sum() / df['n'].sum()) * 100
    df.loc[len(df)] = ['', 'TOTAL', f"{df['n'].sum():_}", f"{df['wins'].sum():_}", f'{avg:.3f}%', df['net'].sum()]
    df['net'] = [locale.currency(s, grouping=True) for s in df['net'].values]
    df['n'] = [f'{int(a):_}' for a in df['n']]
    df['wins'] = [f'{int(a):_}' for a in df['wins']]
    df.index = list(range(len(df)))
    return df

In [12]:
# A simple context manager to time code execution inside a with statement

from contextlib import contextmanager
import time

@contextmanager
def timer():
    before = time.time()
    yield
    after = time.time()
    print(f'Block executed in {after - before:.2f} seconds')

In [13]:
# This cell is not necessary but it is nice to see the draw and corresponding animals before the simulation

lottery_draw = random_draw()
animal_names = [ANIMALS[int(animal_id_from_digits(n))] for n in lottery_draw]
print(animal_names)
lottery_draw

['Cachorro', 'Carneiro', 'Urso', 'Leão', 'Cachorro']


['8118', '2926', '6892', '2961', '9217']

In [14]:
# Make the simulation and time the execution

with timer():
    sim = make_simulation(draw=lottery_draw, bet_mean=11, bet_std=5, size=1_001_000, house_pov=house_pov).sort_values('net')

Block executed in 50.74 seconds


In [15]:
# Sim display is used to veiw currency in a more friendly format than native floats

sim_display = sim.head(12).copy()
sim_display['net'] = [locale.currency(n, grouping=True) for n in sim_display['net'].values]
sim_display['bet'] = [locale.currency(b, grouping=True) for b in sim_display['bet'].values]
sim_display

,bet,game_type,code,picks,net,draw
23558,"R$ 21,20",Milhar Seco,MS,[8118],"-R$ 84.778,80","[8118, 2926, 6892, 2961, 9217]"
517400,"R$ 15,10",Milhar Seco,MS,[8118],"-R$ 60.384,90","[8118, 2926, 6892, 2961, 9217]"
476393,"R$ 19,60",Dezena Terno,DT,"[92, 18, 17]","-R$ 58.780,40","[8118, 2926, 6892, 2961, 9217]"
940786,"R$ 18,20",Dezena Terno,DT,"[17, 92, 61]","-R$ 54.581,80","[8118, 2926, 6892, 2961, 9217]"
901566,"R$ 13,60",Milhar Seco,MS,[8118],"-R$ 54.386,40","[8118, 2926, 6892, 2961, 9217]"
245798,"R$ 16,20",Dezena Terno,DT,"[17, 26, 92]","-R$ 48.583,80","[8118, 2926, 6892, 2961, 9217]"
978522,"R$ 14,90",Dezena Terno,DT,"[18, 92, 17]","-R$ 44.685,10","[8118, 2926, 6892, 2961, 9217]"
277906,"R$ 13,80",Dezena Terno,DT,"[92, 26, 17]","-R$ 41.386,20","[8118, 2926, 6892, 2961, 9217]"
747883,"R$ 9,30",Milhar Seco,MS,[8118],"-R$ 37.190,70","[8118, 2926, 6892, 2961, 9217]"
257559,"R$ 10,20",Dezena Terno,DT,"[18, 61, 26]","-R$ 30.589,80","[8118, 2926, 6892, 2961, 9217]"


In [16]:
with timer():
    by_type = regroup_by_type(source=sim, house_pov=house_pov)
by_type

Block executed in 0.42 seconds


,code,game_type,n,wins,p,net
0,GS,Grupo Seco,82_247,3_347,4.069%,"R$ 248.809,00"
1,CQ,Centena Qualquer,81_931,446,0.544%,"R$ 317.437,10"
2,CS,Centena Seca,82_876,93,0.112%,"R$ 343.200,40"
3,DS,Dezena Seca,82_487,837,1.015%,"R$ 362.842,60"
4,GQ,Grupo Qualquer,81_898,13_681,16.705%,"R$ 364.320,72"
5,DQ,Dezena Qualquer,82_507,4_084,4.950%,"R$ 367.851,80"
6,DD,Duque Dezenas,82_691,171,0.207%,"R$ 379.644,00"
7,MQ,Milhar Qualquer,81_722,53,0.065%,"R$ 409.896,40"
8,GD,Grupo Duque,82_429,1_839,2.231%,"R$ 542.440,00"
9,DT,Dezena Terno,82_509,7,0.008%,"R$ 617.206,70"


In [17]:
hr = (sim['net'].sum() / sim['bet'].sum()) * 100
f'House return on bets: {hr:.2f}%'

'House return on bets: 47.44%'

In [18]:
last_type = GameMeta.get_types()[-1]
mask = sim['code'] == last_type.name
games = sim[mask].head(8).copy()
games['net'] = [locale.currency(n, grouping=True) for n in games['net'].values]
games['bet'] = [locale.currency(b, grouping=True) for b in games['bet'].values]
games

,bet,game_type,code,picks,net,draw
23558,"R$ 21,20",Milhar Seco,MS,[8118],"-R$ 84.778,80","[8118, 2926, 6892, 2961, 9217]"
517400,"R$ 15,10",Milhar Seco,MS,[8118],"-R$ 60.384,90","[8118, 2926, 6892, 2961, 9217]"
901566,"R$ 13,60",Milhar Seco,MS,[8118],"-R$ 54.386,40","[8118, 2926, 6892, 2961, 9217]"
747883,"R$ 9,30",Milhar Seco,MS,[8118],"-R$ 37.190,70","[8118, 2926, 6892, 2961, 9217]"
275845,"R$ 4,50",Milhar Seco,MS,[8118],"-R$ 17.995,50","[8118, 2926, 6892, 2961, 9217]"
651279,"R$ 4,50",Milhar Seco,MS,[8118],"-R$ 17.995,50","[8118, 2926, 6892, 2961, 9217]"
956972,"R$ 0,10",Milhar Seco,MS,[5787],"R$ 0,10","[8118, 2926, 6892, 2961, 9217]"
672866,"R$ 0,10",Milhar Seco,MS,[7865],"R$ 0,10","[8118, 2926, 6892, 2961, 9217]"


In [19]:
mask = sim['code'] == 'DT'
frame = sim[mask].copy()
frame['net'] = [locale.currency(s, grouping=True) for s in frame['net'].values]
frame['bet'] = [locale.currency(b, grouping=True) for b in frame['bet'].values]
frame.head(12)

,bet,game_type,code,picks,net,draw
476393,"R$ 19,60",Dezena Terno,DT,"[92, 18, 17]","-R$ 58.780,40","[8118, 2926, 6892, 2961, 9217]"
940786,"R$ 18,20",Dezena Terno,DT,"[17, 92, 61]","-R$ 54.581,80","[8118, 2926, 6892, 2961, 9217]"
245798,"R$ 16,20",Dezena Terno,DT,"[17, 26, 92]","-R$ 48.583,80","[8118, 2926, 6892, 2961, 9217]"
978522,"R$ 14,90",Dezena Terno,DT,"[18, 92, 17]","-R$ 44.685,10","[8118, 2926, 6892, 2961, 9217]"
277906,"R$ 13,80",Dezena Terno,DT,"[92, 26, 17]","-R$ 41.386,20","[8118, 2926, 6892, 2961, 9217]"
257559,"R$ 10,20",Dezena Terno,DT,"[18, 61, 26]","-R$ 30.589,80","[8118, 2926, 6892, 2961, 9217]"
926564,"R$ 9,40",Dezena Terno,DT,"[92, 61, 17]","-R$ 28.190,60","[8118, 2926, 6892, 2961, 9217]"
875285,"R$ 0,10",Dezena Terno,DT,"[51, 25, 64]","R$ 0,10","[8118, 2926, 6892, 2961, 9217]"
368463,"R$ 0,10",Dezena Terno,DT,"[91, 57, 19]","R$ 0,10","[8118, 2926, 6892, 2961, 9217]"
728454,"R$ 0,10",Dezena Terno,DT,"[02, 06, 96]","R$ 0,10","[8118, 2926, 6892, 2961, 9217]"


In [20]:
animal_names

['Cachorro', 'Carneiro', 'Urso', 'Leão', 'Cachorro']